In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import fft

Data = np.loadtxt('TimeData.txt')


In [4]:
print(Data)

[[-0.85142   0.240153 -0.183659 ... -0.357494 -0.621651  0.356824]
 [ 0.291094  0.160894 -0.073074 ...  0.48705   0.30584   0.555318]
 [ 0.582921  0.119766 -0.936122 ...  0.152924 -0.213712 -0.143682]
 ...
 [-1.499461  0.443188  1.126552 ...  0.440812  0.050718 -1.059766]
 [ 1.124024  0.807586 -1.254455 ... -0.959404  0.949614 -0.435874]
 [ 0.440219  0.437777  0.441441 ... -0.489821 -0.490432 -0.498386]]


In [2]:
df = abs(fft.fft(Data)[:][:4096])/4096
plt.plot(df[120][100:4096])

plt.show()

array([[8.09233887e-04, 2.99049792e-01, 1.11078097e-01, ...,
        5.24183781e-02, 1.11078097e-01, 2.99049792e-01],
       [7.97940918e-04, 2.03210972e-01, 1.40301964e-01, ...,
        4.40538233e-02, 1.40301964e-01, 2.03210972e-01],
       [9.68770996e-04, 2.01854460e-01, 4.77025757e-02, ...,
        8.76813551e-03, 4.77025757e-02, 2.01854460e-01],
       ...,
       [7.77459473e-04, 2.09875371e-01, 1.02915668e-01, ...,
        7.23838958e-02, 1.02915668e-01, 2.09875371e-01],
       [6.90795898e-06, 2.38931955e-01, 1.18684455e-01, ...,
        8.05697261e-02, 1.18684455e-01, 2.38931955e-01],
       [3.07031982e-04, 3.00257408e-01, 1.47404699e-01, ...,
        9.86202936e-02, 1.47404699e-01, 3.00257408e-01]])